https://towardsdatascience.com/creating-interactive-maps-for-instagram-with-python-and-folium-68bc4691d075

In [7]:
import folium
import pandas as pd
import json
from shapely.geometry import shape, Point
from bs4 import BeautifulSoup
from folium.plugins import LocateControl, MarkerCluster

In [17]:
# Create the map
MyMap = folium.Map(location=[52.51685, 13.320943], 
                   tiles='Stamen Terrain', 
                   zoom_start=4, 
                   max_zoom=19, 
                   control_scale=True)

#location, coordinates of the centre of the map
#tiles, open source map tiles Open street map (OSM)
## https://leaflet-extras.github.io/leaflet-providers/preview/
# contro scale, adds a scale bar


# Save the map
MyMap.save('MyMap.html')

In [18]:
# Create custom beer glass icon for map markers
LOC_icon_img = 'icon.png'

# Load in Oxford City Boundary json and create a Shapely 'shape' geometry from it.
#with open('./Images/Oxford City Boundary.json', 'r') as file:
#    oxford_json = json.load(file)
#oxford_boundary = shape(oxford_json['features'][0]['geometry'])

# Create marker cluster to help define cluster of Oxford pubs. Cluster will separate into individual markers at zoom level 13.
#oxford_cluster = MarkerCluster(options={'showCoverageOnHover': False,
#                                        'zoomToBoundsOnClick': True,
#                                        'spiderfyOnMaxZoom': False,
#                                        'disableClusteringAtZoom': 13})

In [19]:
df = pd.read_csv(r'visited.csv', index_col=0, encoding='windows-1252')
df["Coordinates"]= df.apply(lambda row: (row.lat, row.long), axis=1)  #Create coordinates list
df = df.drop(columns=["Country", "lat","long"]).T
df.reset_index(drop=True, inplace=True)
print (df)

City                                  Mt. Etna  \
0     https://www.instagram.com/p/B26ZzpUCZk4/   
1                            (37.751, 14.9934)   

City                                      Doha  \
0     https://www.instagram.com/p/B5oX9rNAf8K/   
1                            (25.2854, 51.531)   

City                                  Salzburg  \
0     https://www.instagram.com/p/B6Abjc2gpNv/   
1                            (47.8095, 13.055)   

City                                  Chemnitz  \
0     https://www.instagram.com/p/B7RTsYiiQwp/   
1                           (50.8357, 12.9292)   

City                                 Tegernsee  \
0     https://www.instagram.com/p/B9cSEwBni4c/   
1                           (47.7051, 11.7647)   

City                                    Athens  \
0     https://www.instagram.com/p/Ba1QQpSFgru/   
1                           (37.9838, 23.7275)   

City                                 Santorini  \
0     https://www.instagram.com/p/Ba3AsnVFAj

In [20]:
places_visited_2= df.to_dict()

In [21]:
# Create markers for each pub in the pub dictionary
for plac, details in places_visited_2.items():
    # Define marker variables
    name = plac
    coordinates = details[1]
    insta_post = details[0]
    #website = details[2]
    #directions = details[3]

    # Create custom icon with beer glass image
    custom_icon = folium.CustomIcon(LOC_icon_img, icon_size=(35, 35), popup_anchor=(0, -22))
    # Define html inside marker pop-up
    pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;">{name}</span></p>
    <p style="text-align: center;"><iframe src={insta_post}embed width="240" height="290" frameborder="0" scrolling="auto" allowtransparency="true"></iframe>
    """, script=True)
    # Create pop-up with html content
    popup = folium.Popup(pub_html, max_width=700)
    # Create marker with custom icon and pop-up.
    custom_marker = folium.Marker(location=coordinates, icon=custom_icon, tooltip=name, popup=popup)
    
    # If pub is within Oxford boundary, add to Oxford marker cluster
    #if oxford_boundary.contains(Point((coordinates[1], coordinates[0]))):
    #    custom_marker.add_to(oxford_cluster)
    #else:
    #    # Else add marker to map
    custom_marker.add_to(MyMap)

# Add oxford cluster to the map
#oxford_cluster.add_to(MyMap)

# Enable geolocation button on map.
LocateControl(auto_start=False).add_to(MyMap)

# Define webpage title html and add to script.
tab_title = """<title>My places Maps</title>"""
MyMap.get_root().html.add_child(folium.Element(tab_title))

# Save map to HTML
MyMap.save('MyMap.html')